In [5]:
import os
import sys
code_path = r"D:\DATN\QA_System\code"
sys.path.append(code_path)
# Import trực tiếp từ các file
from utils import convert_df_to_documents
from model_config import load_gemini15 , load_tiktoken, load_tokenizer
import os
import pandas as pd
from dotenv import load_dotenv
import time
load_dotenv()

True

In [6]:
from milvus_db import MilvusDB, MilvusQADB
from pymilvus import (
    Collection,
    CollectionSchema,
    DataType,
    FieldSchema,
    connections,
    utility,
    MilvusException,
)
from typing import List, Dict

In [8]:
# Kết nối Milvus
def connect_to_milvus(uri="http://localhost:19530"):
    try:
        connections.connect(uri=uri)
        print("[INFO] Connected to Milvus.")
        print("list_collections: ", utility.list_collections())
    except MilvusException as e:
        print(f"[ERROR] Error connecting to Milvus: {e}")
        raise e

# Hàm kiểm tra trạng thái collection
def check_collection_status(col_name: str):
    try:
        if utility.has_collection(col_name):
            collection = Collection(name=col_name)
            print(f"[INFO] Collection '{col_name}' exists. Total entities: {collection.num_entities}")
        else:
            print(f"[INFO] Collection '{col_name}' does not exist.")
    except MilvusException as e:
        print(f"[ERROR] Error checking collection '{col_name}': {e}")
        raise e
# Main logic
if __name__ == "__main__":
    connect_to_milvus()

    # Tên collection
    col_name = "raptor"

    # Kiểm tra trạng thái collection
    check_collection_status(col_name)
    check_collection_status(col_name="base_qa")

[INFO] Connected to Milvus.
list_collections:  ['hybrid_demo', 'noraptor', 'base_qa', 'raptor']
[INFO] Collection 'raptor' exists. Total entities: 1198
[INFO] Collection 'base_qa' exists. Total entities: 2


In [9]:
from milvus_db import MilvusDB, MilvusQADB# step4_vector_store
from utils import convert_df_to_documents
from model_config import VietnameseEmbeddings
import os
import pandas as pd
def load_final_df():
    final_df_path = r"D:\DATN\QA_System\data_analyze\finaldf.pkl"
    if os.path.exists(final_df_path):
        final_df = pd.read_pickle(final_df_path)
        print(f"Đã load final_df từ file: {final_df_path}")
        return final_df
    else:
        print("File final_df chưa tồn tại.")
        return None

final_df = load_final_df()
documents = convert_df_to_documents(final_df)
corpus=[doc.page_content for doc in documents]

Đã load final_df từ file: D:\DATN\QA_System\data_analyze\finaldf.pkl
Processed 1198 out of 1198 documents.


In [13]:
utility.list_collections()

['noraptor', 'base_qa', 'raptor', 'hybrid_demo']

In [14]:
check_collection_status("noraptor")
check_collection_status("raptor")

[INFO] Collection 'noraptor' exists. Total entities: 981
[INFO] Collection 'raptor' exists. Total entities: 1198


In [12]:
milvus_db = MilvusDB("noraptor", corpus=corpus)
milvus_db.load_collection()
import pandas as pd
from tqdm import tqdm

def evaluate_hit_and_mrr(file_path, milvus_db, top_k=5, output_file="evaluation_results.xlsx"):
    """
    Evaluate hit@k and MRR@k for the retrieval system and save results to an Excel file.
    
    :param file_path: Path to the Excel file containing metadata, text, question, and answer.
    :param milvus_db: Vector database object with a perform_retrieval(query, top_k) method.
    :param top_k: Number of top results to consider for hit@k and MRR@k.
    :param output_file: Path to the output Excel file to save the results.
    :return: Dictionary with hit@k and MRR@k scores.
    """
    # Load the Excel file
    df = pd.read_excel(file_path)
    
    # Initialize counters for hit@k and MRR@k
    hits = 0
    reciprocal_ranks = []
    results = []  # To store results for each question
    
    # Evaluate for each question
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        question = row["question"]
        correct_metadata = row["metadata"]
        
        # Retrieve top_k documents from the database
        relevant_docs = milvus_db.perform_retrieval(query=question, top_k=top_k)
        retrieved_metadata = [doc.metadata.get("source") for doc in relevant_docs]
        
        # Check if the correct metadata is in the top_k results
        if correct_metadata in retrieved_metadata:
            hits += 1
            rank = retrieved_metadata.index(correct_metadata) + 1
            reciprocal_ranks.append(1 / rank)
        else:
            reciprocal_ranks.append(0)
        
        # Store result for each question
        results.append({
            "question": question,
            "correct_metadata": correct_metadata,
            "retrieved_metadata": retrieved_metadata[:top_k],
            "hit@k": 1 if correct_metadata in retrieved_metadata else 0,
            "reciprocal_rank": 1 / (retrieved_metadata.index(correct_metadata) + 1) if correct_metadata in retrieved_metadata else 0
        })
    
    # Calculate hit@k and MRR@k
    hit_at_k = hits / len(df)
    mrr_at_k = sum(reciprocal_ranks) / len(df)
    
    # Save the detailed results and overall metrics to an Excel file
    result_df = pd.DataFrame(results)
    result_df.to_excel(output_file, index=False)
    
    # Return overall metrics
    return {"hit@k": hit_at_k, "mrr@k": mrr_at_k}


# Example usage
file_path = r"D:\DATN\QA_System\eval\testset\final_dataset_qa.xlsx"
top_k = 5
output_file = r"D:\DATN\QA_System\eval\testset\evaluation_results5.xlsx"  # Specify the output file path

scores = evaluate_hit_and_mrr(file_path, milvus_db, top_k=top_k, output_file=output_file)
print(f"Hit@{top_k}: {scores['hit@k']}")
print(f"MRR@{top_k}: {scores['mrr@k']}")


100%|██████████| 2560/2560 [13:16<00:00,  3.21it/s]


Hit@5: 0.44453125
MRR@5: 0.21320963541666668


In [21]:
from pymilvus.model.sparse.bm25.tokenizers import build_default_analyzer
analyzer = build_default_analyzer(language="en")

In [ ]:
from datasets import Dataset